In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'handlers/')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/handlers/session_handler.py"
"""
Session handler for the music recommender app.
Manages user sessions and music interactions.
"""
import sys
import importlib
import time
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# Import modules with reload to avoid circular dependencies
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import session_state
importlib.reload(session_state)
from session_state import session_state

sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/final/app')
import navigation
importlib.reload(navigation)
from navigation import navigate_to, show_loading, show_status, get_output

class SessionHandler:
    """Handler for user sessions and music interactions."""

    def __init__(self, recommendation_service, emotion_service, logging_service):
        """
        Initialize the session handler.

        Args:
            recommendation_service: Instance of RecommendationService
            emotion_service: Instance of EmotionService
            logging_service: Instance of LoggingService
        """
        self.recommendation_service = recommendation_service
        self.emotion_service = emotion_service
        self.logging_service = logging_service

    def change_mode(self, mode):
        """
        Change the session mode.

        Args:
            mode (str): The new mode ('none', 'log', 'recommend')

        Returns:
            str: Description of the selected mode
        """
        session_state.mode = mode

        # Mode descriptions
        mode_descriptions = {
            'none': "Private Mode: Your activity won't be logged or used for recommendations.",
            'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
            'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
        }

        show_status(f"Mode changed to: {self.get_mode_label(mode)}", 'info')

        return mode_descriptions[mode]

    def get_mode_label(self, mode):
        """
        Get a human-readable label for a mode.

        Args:
            mode (str): The mode

        Returns:
            str: Human-readable label
        """
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }
        return mode_labels.get(mode, 'Unknown Mode')

    def play_random_song(self):
        """
        Play a random song.

        Returns:
            str: Status message
        """
        show_loading("Finding a song...")

        if session_state.mode == 'none':
            return "Privacy mode → not logging song play."

        song = self.recommendation_service.get_random_song()
        if song is None:
            return "No songs available."

        mood_before = self.emotion_service.detect(show_ui=False)
        mood_after = self.emotion_service.detect(show_ui=False)

        self.logging_service.log(
            session_state.user_id,
            song['song_id'],
            mood_before,
            mood_after,
            is_recommended=0
        )

        time.sleep(1)  # Reduced delay for better responsiveness
        return f"Now playing: {song['artist']} - {song['song']}"

    def handle_sad_mood(self):
        """
        Handle user indicating they feel sad.

        Returns:
            tuple: (status_message, song_selection_widgets or None)
        """
        if session_state.mode != 'recommend':
            return "Not in recommend mode — please switch to 'Recommend Me Music' mode first.", None

        show_loading("Finding songs to lift your mood...")
        recommended = self.recommendation_service.recommend_songs(session_state.user_id)

        if recommended.empty:
            return "No songs found to recommend. Try adding more favorite artists.", None

        options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

        # Create song selection UI
        song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
        song_options = widgets.RadioButtons(
            options=options,
            layout=widgets.Layout(width='auto', margin='10px auto')
        )
        confirm_button = widgets.Button(
            description='Play Selected Song',
            button_style='success',
            layout=widgets.Layout(margin='10px auto', width='auto', min_width='150px')
        )

        def on_confirm_click(_):
            """Handle play selected song button click"""
            if not song_options.value:
                show_status("Please select a song first", 'warning')
                return

            show_loading("Playing your selection...")
            selected_label = song_options.value
            selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
            mood_before = 'sad'
            mood_after = self.emotion_service.detect(show_ui=False)
            self.logging_service.log(
                session_state.user_id,
                selected_row['song_id'],
                mood_before,
                mood_after,
                is_recommended=1
            )
            time.sleep(1)
            navigate_to('session')
            show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

        confirm_button.on_click(on_confirm_click)

        song_ui = widgets.VBox([
            song_header,
            song_options,
            confirm_button
        ], layout=widgets.Layout(align_items='center', width='100%', max_width='500px', margin='0 auto'))

        return None, song_ui

    def handle_happy_mood(self):
        """
        Handle user indicating they feel happy.

        Returns:
            str: Status message
        """
        return "Thanks for sharing! We're glad you're feeling happy today. 😊"

    def logout(self):
        """
        Log the user out.

        Returns:
            None
        """
        session_state.logout()
        navigate_to('start')

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/handlers/session_handler.py
